Data from https://www.kaggle.com/daavoo/3d-mnist#full_dataset_vectors.h5

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import sys; sys.path.append('..')
import fastai_scans, bcolz, h5py
from fastai.basics import *

path = Path('../data')

In [2]:
file = h5py.File(path/'full_dataset_vectors.h5')

# We can directly put the whole file on bcolz format

In [3]:
x_train = bcolz.carray(file['X_train'][:].reshape([-1,16,16,16]).astype(np.float32),
                       chunklen=1, mode='w', rootdir=path/'x_train')
x_train.flush()
x_train.shape

(10000, 16, 16, 16)

In [ ]:
y_train = pd.DataFrame({'label': file['y_train'][:].astype(np.long)})
y_train.to_csv(path/'y_train.csv', index=False)
y_train.head(3)

# But in most cases we will like to read files one by one

In [5]:
def read(idx):
    x = file['X_train'][idx].reshape([16,16,16]).astype(np.float32)
    y = file['y_train'][idx].astype(np.long)
    return x,y

In [6]:
x_train = bcolz.carray(np.zeros([0,16,16,16], dtype=np.float32), chunklen=1, mode='w', rootdir=path/'x_train')
y_train = []

for idx in progress_bar(range(10_000)):
    x,y = read(idx)
    x_train.append(x)
    y_train.append(y)

y_train = pd.DataFrame({'label': y_train})
y_train.to_csv(path/'y_train.csv', index=False)
x_train.flush()
x_train.shape, y_train.shape

((10000, 16, 16, 16), (10000, 1))

# Reading data

In [7]:
x_train = bcolz.open(path/'x_train', mode='r')
x_train.shape

(10000, 16, 16, 16)

In [9]:
y_train = pd.read_csv(path/'y_train.csv')
y_train.shape

(10000, 1)